# Sistema Antifraude com Análise de Dados Transacionais

Este notebook segue o plano de análise para investigar fraudes em transações móveis e construir um sistema antifraude simples baseado em regras e/ou machine learning.

In [ ]:
# Bibliotecas principais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Visualização
sns.set(style='whitegrid')

## 1. Leitura dos dados

In [ ]:
# Substitua pelo caminho do seu CSV
df = pd.read_csv('dados_transacoes.csv')
df.head()

## 2. Pré-processamento dos dados

In [ ]:
# Verificando nulos e tipos de dados
print(df.info())
print("\nNulos:\n", df.isnull().sum())

# Convertendo datas
if df['transaction_date'].dtype == 'object':
    df['transaction_date'] = pd.to_datetime(df['transaction_date'])

In [ ]:
# Removendo duplicatas e dados inconsistentes
df = df.drop_duplicates()
df = df[df['transaction_amount'] > 0]

# Normalizando colunas de texto se necessário (ex: card_number)
df['card_number'] = df['card_number'].astype(str).str.replace(r'\D', '', regex=True)

## 3. Análise Exploratória

In [ ]:
# Estatísticas gerais
print(df.describe())
print("\nTransações com estorno:", df['has_cbk'].sum())

In [ ]:
# Distribuição temporal
df['hour'] = df['transaction_date'].dt.hour
df['day'] = df['transaction_date'].dt.date

plt.figure(figsize=(10,4))
sns.countplot(x='hour', data=df, hue='has_cbk')
plt.title('Transações por Hora com Estornos')
plt.show()

## 4. Regras Antifraude Simples

In [ ]:
# Sistema antifraude baseado em regras simples
def avaliar_transacao(transacao, historico):
    user_transacoes = historico[historico['user_id'] == transacao['user_id']]

    # Regra 1: Mais de 5 transações na última hora
    recentes = user_transacoes[user_transacoes['transaction_date'] > transacao['transaction_date'] - timedelta(hours=1)]
    if len(recentes) > 5:
        return "recusar"

    # Regra 2: Limite diário > R$5000
    dia = transacao['transaction_date'].date()
    total_dia = user_transacoes[user_transacoes['transaction_date'].dt.date == dia]['transaction_amount'].sum()
    if total_dia + transacao['transaction_amount'] > 5000:
        return "recusar"

    # Regra 3: Usuário com histórico de estorno
    if user_transacoes['has_cbk'].any():
        return "recusar"

    return "aprovar"

## 5. Simulação de avaliação de transações

In [ ]:
# Exemplo: simular decisões
resultados = []

for i, row in df.iterrows():
    decisao = avaliar_transacao(row, df[df['transaction_date'] < row['transaction_date']])
    resultados.append(decisao)

df['recomendacao'] = resultados

# Comparando com estornos reais
df[['recomendacao', 'has_cbk']].value_counts(normalize=True)

## 6. Conclusões e próximos passos

In [ ]:
# Exemplo de recomendação de melhoria
print("Sugestão: adicionar geolocalização e device_info para melhorar o score de risco.")